# Lab - SQL 

## Task 0 - data ingestion 

### check the sakila databse

In [ ]:
import duckdb 

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC;").df() 

description

### Storing data into dictionary of dfs()

In [15]:
dfs = {}

with duckdb.connect("data/sakila.duckdb") as conn: 
    for name in description["name"]: 
        #exclude if name starts with _  
        if name[0] != '_':
            dfs[name] = conn.sql(f"FROM staging.{name};").df()

            #register into duckdb database in the temp database  
            duckdb.register(name, dfs[name])

 

In [20]:
duckdb.sql("DESC;").df()

,database,schema,name,column_names,column_types,temporary
0,temp,main,actor,"[actor_id, first_name, last_name, last_update,...","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP, VARCHAR,...",True
1,temp,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, ""NULL"", VARCHAR, BIGINT, VAR...",True
2,temp,main,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",True
3,temp,main,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",True
4,temp,main,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",True
5,temp,main,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",True
6,temp,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",True
7,temp,main,film_actor,"[actor_id, film_id, last_update, _dlt_load_id,...","[BIGINT, BIGINT, TIMESTAMP, VARCHAR, VARCHAR]",True
8,temp,main,film_category,"[film_id, category_id, last_update, _dlt_load_...","[BIGINT, BIGINT, TIMESTAMP, VARCHAR, VARCHAR]",True
9,temp,main,inventory,"[inventory_id, film_id, store_id, last_update,...","[BIGINT, BIGINT, BIGINT, TIMESTAMP, VARCHAR, V...",True


## Task 1 - EDA in python

#### A) - Which movies are longer than 3 hours (180 minutes), show the title and its length?

- By studying the Sakila database documentation I know that the lenght is in minutes 

In [48]:
movie_lenght = duckdb.sql(
"""
--sql
SELECT
    title, 
    length 
FROM main.film
WHERE length > 180
ORDER BY length desc;
""")

movie_lenght.df()

,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


#### b) Which movies have the word "love" in its title? Show the following columns
- title
- rating
- length
- description

In [45]:
word_in_movie_love = duckdb.sql("""
--sql
SELECT 
    title,
    rating, 
    length, 
    description                             
FROM main.film
WHERE title LIKE '%LOVE'; 
;
""")

word_in_movie_love.df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...


#### c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

In [ ]:
film_length_values = duckdb.sql("""
--sql
SELECT 
    MAX(length) as max_length,
    MIN(length) as min_length,
    ROUND(AVG(length)) as avg_length, 
    MEDIAN(length) as median_length
;
""")

film_length_values.df()

BinderException: Binder Error: Referenced column "length" was not found because the FROM clause is missing